In [1]:
import numpy as np
import tensorflow as tf

In [4]:
features = [tf.contrib.layers.real_valued_column("x", dimension=1)]

In [6]:
estimator = tf.contrib.learn.LinearRegressor(feature_columns=features)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x11f866ef0>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/var/folders/8n/7b6wkcdx00x0vkf6pbr7p8n80000gp/T/tmpaowh8g6c'}


In [7]:
x_train = np.array([1.,2.,3.,4.])
y_train = np.array([0.,-1.,-2.,-3.])
x_eval = np.array([2.,5.,8.,1.])
y_eval = np.array([-1.01,-4.1,-7,0.])

In [8]:
input_fn = tf.contrib.learn.io.numpy_input_fn({"x":x_train}, y_train, batch_size=4,num_epochs=1000)

In [9]:
eval_input_fn = tf.contrib.learn.io.numpy_input_fn({"x":x_eval}, y_eval,batch_size=4,num_epochs=1000)

In [10]:
estimator.fit(input_fn=input_fn,steps=1000)

Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /var/folders/8n/7b6wkcdx00x0vkf6pbr7p8n80000gp/T/tmpaowh8g6c/model.ckpt.
INFO:tensorflow:loss = 5.0, step = 1
INFO:tensorflow:global_step/sec: 1125.52
INFO:tensorflow:loss = 0.0656109, step = 101 (0.091 sec)
INFO:tensorflow:global_step/sec: 1266.9
INFO:tensorflow:loss = 0.0254874, step = 201 (0.077 sec)
INFO:tensorflow:global_step/sec: 963.596
INFO:tensorflow:loss = 0.00200801, step = 301 (0.104 sec)
INFO:tensorflow:global_step/sec: 1025.69
INFO:tensorflow:loss = 0.00111278, step = 401 (0.098 sec)
INFO:tensorflow:global_step/sec: 1105.18
INFO:tensorflow:loss = 0.000354704, s

LinearRegressor(params={'head': <tensorflow.contrib.learn.python.learn.estimators.head._RegressionHead object at 0x11f866b00>, 'feature_columns': [_RealValuedColumn(column_name='x', dimension=1, default_value=None, dtype=tf.float32, normalizer=None)], 'optimizer': None, 'gradient_clip_norm': None, 'joint_weights': False})

In [11]:
train_loss = estimator.evaluate(input_fn=input_fn)

Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
INFO:tensorflow:Starting evaluation at 2017-07-31-03:18:58
INFO:tensorflow:Restoring parameters from /var/folders/8n/7b6wkcdx00x0vkf6pbr7p8n80000gp/T/tmpaowh8g6c/model.ckpt-1000
INFO:tensorflow:Finished evaluation at 2017-07-31-03:18:59
INFO:tensorflow:Saving dict for global step 1000: global_step = 1000, loss = 1.87911e-07


In [12]:
eval_loss = estimator.evaluate(input_fn=eval_input_fn)

Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
INFO:tensorflow:Starting evaluation at 2017-07-31-03:19:41
INFO:tensorflow:Restoring parameters from /var/folders/8n/7b6wkcdx00x0vkf6pbr7p8n80000gp/T/tmpaowh8g6c/model.ckpt-1000
INFO:tensorflow:Finished evaluation at 2017-07-31-03:19:42
INFO:tensorflow:Saving dict for global step 1000: global_step = 1000, loss = 0.00257326


In [13]:
print("train loss: %r"% train_loss)
print("eval loss: %r"% eval_loss)

train loss: {'loss': 1.8791128e-07, 'global_step': 1000}
eval loss: {'loss': 0.0025732564, 'global_step': 1000}


In [22]:
def model(features, labels, mode):
  # Build a linear model and predict values
  W = tf.get_variable("W", [1], dtype=tf.float64)
  b = tf.get_variable("b", [1], dtype=tf.float64)
  y = W*features['x'] + b
  # Loss sub-graph
  loss = tf.reduce_sum(tf.square(y - labels))
  # Training sub-graph
  global_step = tf.train.get_global_step()
  optimizer = tf.train.GradientDescentOptimizer(0.01)
  train = tf.group(optimizer.minimize(loss),
                   tf.assign_add(global_step, 1))
  # ModelFnOps connects subgraphs we built to the
  # appropriate functionality.
  return tf.contrib.learn.ModelFnOps(
      mode=mode, predictions=y,
      loss=loss,
      train_op=train)

In [24]:
estimator = tf.contrib.learn.Estimator(model_fn=model)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1200f3b38>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/var/folders/8n/7b6wkcdx00x0vkf6pbr7p8n80000gp/T/tmpty8ifw85'}


In [17]:
x_train = np.array([1.,2.,3.,4.])
y_train = np.array([0.,-1.,-2.,-3.])
x_eval = np.array([2.,5.,8.,1.])
y_eval = np.array([-1.01,-4.1,-7,0.])
input_fn = tf.contrib.learn.io.numpy_input_fn({"x":x_train}, y_train, 4, num_epochs = 1000)

In [26]:
# train
estimator.fit(input_fn=input_fn, steps=1000)
# Here we evaluate how well our model did. 
train_loss = estimator.evaluate(input_fn=input_fn)
eval_loss = estimator.evaluate(input_fn=eval_input_fn)
print("train loss: %r"% train_loss)
print("eval loss: %r"% eval_loss)

NameError: name 'minimize' is not defined